In [66]:
import folium
import openrouteservice

from math import cos, asin, sqrt

import pandas as pd

# Dataset

  * [Parcs et jardins de nantes](https://data.nantesmetropole.fr/explore/embed/dataset/244400404_parcs-jardins-nantes/table/?disjunctive.libtype&disjunctive.gardien&disjunctive.jeux_enfants&disjunctive.pataugeoire&disjunctive.sanitaires&disjunctive.sanitaires_handicapes&disjunctive.chiens_autorises&disjunctive.jardin_clos&disjunctive.abris&disjunctive.point_eau_potable&disjunctive.table_pique_nique)

In [70]:
parc_jardin_dataset = pd.read_csv("data/parcs_nantes.csv", low_memory=False,sep=';', header=0, index_col=0)

# Geolocalisation transformation

In [89]:
# https://en.wikipedia.org/wiki/Haversine_formula
# Get distance between 2 points in km
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Math.PI / 180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a)) # 2 * R; R = 6371 km

# Return N closest points
def closest(data, v, number):
    res = []
    for val in data:
        val['distance'] = distance(val['lat'],val['lon'],v['lat'],v['lon'])
    
    data.sort(key=lambda x: x['distance'])
    return data[:number]

# Return N closest points
def closestPanda(data, v, number):
    res = []
    for val in data:
        val['distance'] = distance(val['lat'],val['lon'],v['lat'],v['lon'])
    
    data.sort(key=lambda x: x['distance'])
    return data[:number]

In [90]:
# Transform string to object 
# Example : '47.2079361916, -1.6101650945' -> {'lon': -1.6101650945, 'lat': 47.2079361916}
def strToCoordObj(val):
    tmp = val.split(',')
    return {'lon': float(tmp[1]), 'lat': float(tmp[0])}

# Transform Pandas.series string to list of coords
def seriesToCoordList(series):
    return list(map(strToCoordObj , series.tolist()))


In [91]:
closestV2(seriesToCoordList(parc_jardin_dataset['Géolocalisation']), {'lat':47.209929, 'lon': -1.60304},5)

[{'lon': -1.6101650945, 'lat': 47.2079361916, 'distance': 0.5820444633617999},
 {'lon': -1.61050747321, 'lat': 47.2045486366, 'distance': 0.8222698758007888},
 {'lon': -1.59232998652, 'lat': 47.2078213764, 'distance': 0.8422716699376822},
 {'lon': -1.60835729019, 'lat': 47.2026323604, 'distance': 0.9053349042844381},
 {'lon': -1.59311609514, 'lat': 47.2041151562, 'distance': 0.9899027703227259}]

# Point of Interest

In [105]:
garden_list = seriesToCoordList(parc_jardin_dataset['Géolocalisation'])

# Visualization

In [106]:
map1 = folium.Map(tiles='openstreetmap', location=([47.218371,-1.553621]), zoom_start=12)

for garden in garden_list:
    folium.map.Marker(
        [garden['lat'], garden['lon']],
        icon=folium.Icon(
            color='white',
            icon_color='#1a1aff',
            icon='tree',
            prefix='fa'
        )
    ).add_to(map1)

map1

# Route


In [ ]:
def addParks(df):
    for row in df.itertuples():
        if <something>:
            df.at[row.Index, 'ifor'] = x
        else:
            df.at[row.Index, 'ifor'] = x

        df.loc[row.Index, 'ifor'] = x

In [4]:

coords = ((-1.558395,47.245491),(-1.60304,47.209929))

client = openrouteservice.Client(key='5b3ce3597851110001cf624834d62a68a42243119e718bb87c5c8e7f') # Specify your personal API key
routes = client.directions(coords)

print(routes)

{'routes': [{'summary': {'distance': 7603.1, 'duration': 932.6}, 'segments': [{'distance': 7603.1, 'duration': 932.6, 'steps': [{'distance': 210.4, 'duration': 43.3, 'type': 11, 'instruction': 'Head west', 'name': '-', 'way_points': [0, 8]}, {'distance': 38.9, 'duration': 8.3, 'type': 0, 'instruction': 'Turn left', 'name': '-', 'way_points': [8, 12]}, {'distance': 423.3, 'duration': 101.6, 'type': 7, 'instruction': 'Enter the roundabout and take the 2nd exit onto Boulevard des Tribunes', 'name': 'Boulevard des Tribunes', 'exit_number': 2, 'way_points': [12, 37]}, {'distance': 43.0, 'duration': 12.5, 'type': 7, 'instruction': 'Enter the roundabout and take the 2nd exit onto Boulevard du Petit Port', 'name': 'Boulevard du Petit Port', 'exit_number': 2, 'way_points': [37, 46]}, {'distance': 228.9, 'duration': 26.1, 'type': 5, 'instruction': 'Turn slight right onto Boulevard du Petit Port', 'name': 'Boulevard du Petit Port', 'way_points': [46, 54]}, {'distance': 475.2, 'duration': 61.8, 't

In [50]:
coords = ((-1.558395,47.245491),(-1.60304,47.209929))

client = openrouteservice.Client(key='5b3ce3597851110001cf624834d62a68a42243119e718bb87c5c8e7f') # Specify your personal API key
routes = client.directions(coords)

direction_params = {'coordinates': coords,
                    'profile': 'driving-car', 
                    'format_out': 'geojson',
                    'preference': 'shortest',
                    'geometry': 'true'}


regular_route = client.directions(**direction_params) # Direction request

# Build popup
duration, distance = regular_route['features'][0]['properties']['summary'].values()
print("Duration : " + str(duration))
print("Distance : " + str(distance))

Duration : 6437.4
Distance : 1105.4


In [110]:
coords = ((-1.558395,47.245491),(-1.60304,47.209929))

client = openrouteservice.Client(base_url='http://localhost:9090/ors')
routes = client.directions(coords)

# foot-walking
direction_params = {'coordinates': coords,
                    'profile': 'driving-car', 
                    'format_out': 'geojson',
                    'preference': 'shortest',
                    'geometry': 'true'}

regular_route = client.directions(**direction_params) # Direction request

# Build popup
duration, distance = regular_route['features'][0]['properties']['summary'].values()
print(duration)
print(distance)

6437.1
1108.8


In [56]:
client = openrouteservice.Client(key='5b3ce3597851110001cf624834d62a68a42243119e718bb87c5c8e7f') # Specify your personal API key
# Set up folium map
map1 = folium.Map(tiles='openstreetmap', location=([47.209929,-1.60304]), zoom_start=12)


# Set up the apartment dictionary with real coordinates
apt_dict = {'first': {'location': [-1.60304,47.209929]}}

# Request of isochrones with 15 minute footwalk.
params_iso = {'profile': 'foot-walking',
              'intervals': [900], # 900/60 = 15 mins
              'segments': 900,
              'attributes': ['total_pop'] # Get population count for isochrones
             }

for name, apt in apt_dict.items():
    params_iso['locations'] = [apt['location']] # Add apartment coords to request parameters
    apt['iso'] = client.isochrones(**params_iso) # Perform isochrone request
    folium.features.GeoJson(apt['iso']).add_to(map1) # Add GeoJson to map
    
    folium.map.Marker(list(reversed(apt['location'])), # reverse coords due to weird folium lat/lon syntax
                      icon=folium.Icon(color='lightgray',
                                        icon_color='#cc0000',
                                        icon='home',
                                        prefix='fa',
                                       ),
                      popup=name,
                 ).add_to(map1) # Add apartment locations to map

map1

In [95]:
apt_dict

{'first': {'location': [-1.60304, 47.209929],
  'iso': {'type': 'FeatureCollection',
   'bbox': [-1.566036, -1.63998, 47.233861, 47.194073],
   'features': [{'type': 'Feature',
     'properties': {'group_index': 0,
      'value': 300.0,
      'center': [-1.6030305727001795, 47.20996861835771],
      'total_pop': 60315.0,
      'style': {},
      'highlight': {}},
     'geometry': {'coordinates': [[[-1.63998, 47.214556],
        [-1.639642, 47.213809],
        [-1.639567, 47.213724],
        [-1.632799, 47.20745],
        [-1.631574, 47.204595],
        [-1.631476, 47.204398],
        [-1.629636, 47.203348],
        [-1.620266, 47.196522],
        [-1.619539, 47.195322],
        [-1.61803, 47.194255],
        [-1.61723, 47.194073],
        [-1.613563, 47.194135],
        [-1.609001, 47.194852],
        [-1.601167, 47.196365],
        [-1.591125, 47.195685],
        [-1.588398, 47.195473],
        [-1.585489, 47.195861],
        [-1.585003, 47.196035],
        [-1.579134, 47.198274],
   

In [108]:
client = openrouteservice.Client(key='5b3ce3597851110001cf624834d62a68a42243119e718bb87c5c8e7f') # Specify your personal API key
# Set up folium map
map1 = folium.Map(tiles='openstreetmap', location=([47.209929,-1.60304]), zoom_start=12)


# Set up the apartment dictionary with real coordinates
apt_dict = {'first': {'location': [-1.60304,47.209929]}}

# Request of isochrones with 5 minute driving.
params_iso = {'profile': 'driving-car',
              'intervals': [300], # 300/60 = 15 mins
              'segments': 300,
              'attributes': ['total_pop'] # Get population count for isochrones
             }

for name, apt in apt_dict.items():
    params_iso['locations'] = [apt['location']] # Add apartment coords to request parameters
    apt['iso'] = client.isochrones(**params_iso) # Perform isochrone request
    folium.features.GeoJson(apt['iso']).add_to(map1) # Add GeoJson to map
    
    folium.map.Marker(list(reversed(apt['location'])), # reverse coords due to weird folium lat/lon syntax
                      icon=folium.Icon(color='lightgray',
                                        icon_color='#cc0000',
                                        icon='home',
                                        prefix='fa',
                                       ),
                      popup=name,
                 ).add_to(map1) # Add apartment locations to map

map1

In [109]:
# Common request parameters
params_poi = {'request': 'pois',
              'sortby': 'distance'}

# POI categories according to 
# https://github.com/GIScience/openrouteservice-docs#places-response
categories_poi = {
    'college': [151],
    'school':[156],
    'university':[157]}
for name, apt in apt_dict.items():
    apt['categories'] = dict() # Store in pois dict for easier retrieval
    params_poi['geojson'] = apt['iso']['features'][0]['geometry']
    print("\n{} apartment".format(name))
    
    for typ, category in categories_poi.items():
        params_poi['filter_category_ids'] = category
        apt['categories'][typ] = dict()
        apt['categories'][typ]['geojson']= client.places(**params_poi)['features'] # Actual POI request
        print("\t{}: {}".format(typ, # Print amount POIs
                                len(apt['categories'][typ]['geojson'])))


first apartment
	college: 6
	school: 46
	university: 3


In [ ]:
# Set up common request parameters
params_route = {'profile': 'foot-walking',
               'format_out': 'geojson',
               'geometry': 'true',
               'format': 'geojson',
               'instructions': 'false',
               }

# Set up dict for font-awesome
style_dict = {'college': 'child',
              'school': 'shopping-cart',
              'university': 'scissors'
             }

# Store all routes from all apartments to POIs
for apt in apt_dict.values():
    for cat, pois in apt['categories'].items():
        pois['durations'] = []
        for poi in pois['geojson']:
            poi_coords = poi['geometry']['coordinates']
            
            # Perform actual request
            params_route['coordinates'] = [apt['location'],
                                           poi_coords
                                          ]
            json_route = clnt.directions(**params_route)
            
            folium.features.GeoJson(json_route).add_to(map1)
            folium.map.Marker(list(reversed(poi_coords)),
                              icon=folium.Icon(color='white',
                                               icon_color='#1a1aff',
                                               icon=style_dict[cat],
                                               prefix='fa'
                                              )
                             ).add_to(map1)
            
            poi_duration = json_route['features'][0]['properties']['summary']['duration']
            pois['durations'].append(poi_duration) # Record durations of routes